In [40]:
import tensorflow as tf

In [41]:
mnist = tf.keras.datasets.mnist

(x_tarin, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_tarin/255., x_test/255.

x_train = tf.expand_dims(x_train[:10000].astype("float32"), axis=-1)
x_test = tf.expand_dims(x_test[:10000].astype("float32"), axis=-1)
y_train, y_test = y_train[:10000], y_test[:10000]

In [42]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [49]:
class MyModel(tf.keras.Model):

    def __init__(self, loss, optimizer):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation="relu")
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation="relu")
        self.dense2 = tf.keras.layers.Dense(10)
    
        self.loss = loss
        self.optimizer = optimizer

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

    def train_step(self, images, labels):
        with tf.GradientTape() as tape:
            predictions = self(images)
            loss = self.loss(labels, predictions)

        gradients = tape.gradient(loss, self.trainable_variables) # compute gradient of loss wrt train vars
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return predictions, loss, gradients

    def test_step(self, images, labels):
        predictions = self(images)
        loss = self.loss(labels, predictions)

        return predictions, loss

In [50]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [51]:
model = MyModel(loss_object, optimizer)

In [52]:
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

In [57]:
EPOCHS = 5

writer = tf.summary.create_file_writer("summary/run_2")

for epoch in range(EPOCHS):
    # Reset the metrics
    train_loss.reset_states()
    test_loss.reset_states()
    train_acc.reset_states()
    test_acc.reset_states()

    for train_images, train_labels in train_ds:
        pred_train, loss_train, grads_train = model.train_step(train_images, train_labels)
        train_loss(loss_train)
        train_acc(train_labels, pred_train)

    for test_images, test_labels in test_ds:
        pred_test, loss_test = model.test_step(test_images, test_labels)
        test_loss(loss_test)
        test_acc(test_labels, pred_test)

        with writer.as_default():
            tf.summary.scalar("Train loss: ", train_loss.result(), step=epoch)
            tf.summary.scalar("\nTest loss: ", test_loss.result(), step=epoch)
            tf.summary.scalar("\nTrain acc: ", train_acc.result(), step=epoch)
            tf.summary.scalar("\nTest acc: ", test_acc.result(), step=epoch)

    message = (
      f"Epoch {epoch + 1}, Train loss: {train_loss.result()}, "
      f"Test loss: {test_loss.result()}, Train acc: {train_acc.result()}"
      f"Test acc: {test_acc.result()}"
    )
    print(message)

Epoch 1, Train loss: 0.002075519412755966, Test loss: 0.1262732595205307, Train acc: 0.9995999932289124Test acc: 0.9684000015258789
Epoch 2, Train loss: 0.0119366105645895, Test loss: 0.12689875066280365, Train acc: 0.9962999820709229Test acc: 0.9664999842643738
Epoch 3, Train loss: 0.011574814096093178, Test loss: 0.15980853140354156, Train acc: 0.9965000152587891Test acc: 0.9634000062942505
Epoch 4, Train loss: 0.004956850782036781, Test loss: 0.12483221292495728, Train acc: 0.9990000128746033Test acc: 0.9707000255584717
Epoch 5, Train loss: 0.00042186968494206667, Test loss: 0.12149380892515182, Train acc: 1.0Test acc: 0.9721999764442444


In [56]:
!tensorboard --logdir summary

2023-08-28 12:04:30.102175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-28 12:04:30.141367: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-28 12:04:30.141813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 12:04:30.735517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-28 12:04:31.580743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do